In [24]:
import boto3
import pandas as pd
import numpy as np
import json
import gzip
import importlib
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
from boto3.dynamodb.types import Binary
from importlib import reload
from spdynamodb import DynamoTable
from time import sleep

In [2]:
dt = DynamoTable()

In [3]:
dt.create_table(table_name="dragon_stats", 
                partition_key="dragon_name", 
                partition_key_type="S", 
                provisioned=False
)

dt.load_json("dragons_dynamo/dragon_stats_one.json")
dt.load_json("dragons_dynamo/dragon_stats_two.json")

Table created successfully!


In [3]:
dt.select_table("dragon_stats")

In [9]:
try: 
    dt.create_table("data_name", "Inv_ID", "S", "Bill_ID", "S")
except:
    dt.select_table("data_name")

Table created successfully!


In [7]:
dt.all_tables

{'Region': 'us-east-1',
 'Tables': [dynamodb.Table(name='Employees'),
  dynamodb.Table(name='SkiLifts'),
  dynamodb.Table(name='Songs'),
  dynamodb.Table(name='dragon_stats')]}

In [14]:
s_out = """Offer for a Basic Digital Access subscription is not open to current digital news s
ubscribers. Your payment method will automatically be charged in advance the introductory 
offer rate every 4 weeks for 1 year, and after 1 year the standard rate every 4 weeks. 
Your subscription will continue until you cancel. Cancellation takes effect at the end 4
of your current billing period. The print edition of The New York Times, Games, Cooking or 
Wirecutter are not included. Taxes may apply. Offer terms are subject to change.
"""
s_out2 = """At its fifth hearing on the Capitol riot, the panel plans to show evidence about how Donald Trump tried to manipulate the Justice Department to stay in power.
The committee is presenting his actions as a critical strand in a multilayered effort by the former president to subvert the election. Watch the hearing live.
"""
s_out3 = "At itsid Watch the hearing live."

item1 = {
    "Inv_ID": "Invoice-40539",
    "Bill_ID": "Bill-235",
    "Data1": s_out,
    "Data2": s_out2
}

item2 = {
    "Inv_ID": "Invoice-40539",
    "Bill_ID": "Bill-15",
    "Data2": s_out3
}

df = pd.DataFrame([item1, item2], index=range(2))

In [15]:
dt.batch_pandas(df, compress=True)

In [16]:
dt.query_items("Invoice-40539", to_pandas=True)

,Data1,Data2,Inv_ID,Bill_ID
0,,At itsid Watch the hearing live.,Invoice-40539,Bill-15
1,Offer for a Basic Digital Access subscription ...,"At its fifth hearing on the Capitol riot, the ...",Invoice-40539,Bill-235


In [18]:
dt.select_table("dragon_stats")

In [21]:
dt

- Table name: dragon_stats            
- Table arn: arn:aws:dynamodb:us-east-1:727741534221:table/dragon_stats            
- Table creation: 2023-03-19 10:29:41.529000-03:00            
- [{'AttributeName': 'dragon_name', 'KeyType': 'HASH'}]            
- [{'AttributeName': 'dragon_name', 'AttributeType': 'S'}]

In [22]:
dt.create_global_secondary_index(
    att_name="family",
    att_type="S",
    i_name="CategoryIndex"
)

In [25]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

Global secondary index created.


In [26]:
dt.check_status_gsi()

'ACTIVE'

In [27]:
result = dt.table.query(IndexName="CategoryIndex",
            KeyConditionExpression=Key('family').eq('red')
        )   

In [30]:
len(result)

4